In [1]:
import argparse
from typing import Dict, Tuple, Any
import json
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import HiveContext
from pyspark.sql.functions import *
import importlib

In [9]:
df_categoria = spark.sql("select * from aula_hive_stg.tbl_categoria")

In [7]:
df_estado = spark.sql("select * from aula_hive_stg.tbl_estado")

In [21]:
df_pedido = spark.sql("select * from aula_hive_stg.tbl_pedido")

In [27]:
df_item_pedido = spark.sql("select * from aula_hive_stg.tbl_item_pedido")

In [30]:
df_item_pedido.printSchema()

root
 |-- id_pedido: string (nullable = true)
 |-- id_produto: string (nullable = true)
 |-- quantidade: string (nullable = true)
 |-- vr_unitario: string (nullable = true)
 |-- dt_foto: string (nullable = true)



In [31]:
df_stage = df_pedido.alias('p').join(df_item_pedido.alias('ip'),df_pedido.id_pedido == df_item_pedido.id_pedido, 'inner')

In [33]:
df_stage.wi

+-----------+--------------------+-----------+----------+---------+-------------+--------+-----------+----------+----------+-----------+--------+
|  id_pedido|           dt_pedido|id_parceiro|id_cliente|id_filial|vr_total_pago| dt_foto|  id_pedido|id_produto|quantidade|vr_unitario| dt_foto|
+-----------+--------------------+-----------+----------+---------+-------------+--------+-----------+----------+----------+-----------+--------+
|46626099016|2021-06-02T00:00:...|         16| 138633690|      547|       194.99|20230603|46626099016|   5077094|         1|     194.99|20230603|
|46630423213|2021-06-02T00:00:...|         13| 106568537|      875|        51.99|20230603|46630423213|   3335368|         1|      51.99|20230603|
|46630859716|2021-06-02T00:00:...|         16| 138757662|      884|        90.99|20230603|46630859716|   5053698|         1|      90.99|20230603|
|46631152013|2021-06-02T00:00:...|         13| 138459547|      494|       129.99|20230603|46631152013|   3488193|         1|

In [35]:
df_pedido.createOrReplaceTempView('tb_pedido')
df_item_pedido.createOrReplaceTempView('tb_item_pedido')

In [41]:
query = '''
select  tb_pedido.id_pedido as nova_coluna, *
from tb_pedido as p
inner join tb_item_pedido as ip
on tb_pedido.id_pedido = tb_item_pedido.id_pedido
'''

df_stage2 = spark.sql(query).show()

+-----------+-----------+--------------------+-----------+----------+---------+-------------+--------+-----------+----------+----------+-----------+--------+
|nova_coluna|  id_pedido|           dt_pedido|id_parceiro|id_cliente|id_filial|vr_total_pago| dt_foto|  id_pedido|id_produto|quantidade|vr_unitario| dt_foto|
+-----------+-----------+--------------------+-----------+----------+---------+-------------+--------+-----------+----------+----------+-----------+--------+
|46626099016|46626099016|2021-06-02T00:00:...|         16| 138633690|      547|       194.99|20230603|46626099016|   5077094|         1|     194.99|20230603|
|46630423213|46630423213|2021-06-02T00:00:...|         13| 106568537|      875|        51.99|20230603|46630423213|   3335368|         1|      51.99|20230603|
|46630859716|46630859716|2021-06-02T00:00:...|         16| 138757662|      884|        90.99|20230603|46630859716|   5053698|         1|      90.99|20230603|
|46631152013|46631152013|2021-06-02T00:00:...|      